In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import urllib.parse

def extract_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    links = [a['href'] for a in soup.find_all('a', href=True)]
    return links

def make_absolute_links(base_url, links):
    absolute_links = []
    for link in links:
        absolute_link = urllib.parse.urljoin(base_url, link)
        absolute_links.append(absolute_link)
    return absolute_links

def save_to_csv(filename, links):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["name", "link"])
        for link in links:
            name = link.split('/')[-1]
            writer.writerow([name, link])

def crawl_website(url):
    links = extract_links(url)
    links = make_absolute_links(url, links)
    save_to_csv("website_links.csv", links)

crawl_website('https://www.usjr.edu.ph')

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import urllib.parse

def extract_links(url, base_url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    links = [a['href'] for a in soup.find_all('a', href=True) if base_url in urllib.parse.urljoin(url, a['href'])]
    return links

def make_absolute_links(base_url, links):
    absolute_links = []
    for link in links:
        absolute_link = urllib.parse.urljoin(base_url, link)
        absolute_links.append(absolute_link)
    return absolute_links

def save_to_csv(filename, links):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["name", "link"])
        for link in links:
            name = link.split('/')[-1]
            writer.writerow([name, link])

def crawl_website(url, base_url):
    visited = set()
    to_visit = [url]
    links = []
    
    while to_visit:
        url = to_visit.pop(0)
        if url not in visited:
            visited.add(url)
            try:
                page_links = extract_links(url, base_url)
                page_links = make_absolute_links(url, page_links)
                links.extend(page_links)
                to_visit.extend(page_links)
            except:
                pass
    
    save_to_csv("website_links.csv", links)

base_url = 'https://www.usjr.com.edu.ph'
crawl_website(base_url, base_url)


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import urllib.parse
from collections import deque

def extract_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    return [a['href'] for a in soup.find_all('a', href=True)]

def is_internal_link(base_url, link):
    return base_url in urllib.parse.urljoin(base_url, link)

def save_to_csv(filename, links):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["name", "link"])
        for link in links:
            name = link.split('/')[-1]
            writer.writerow([name, link])

def crawl_website(base_url):
    visited = set()
    to_visit = deque([base_url])
    internal_links = set()

    while to_visit:
        url = to_visit.popleft()
        if url not in visited:
            visited.add(url)
            print(f"Visiting: {url}")
            try:
                links = extract_links(url)
                for link in links:
                    absolute_link = urllib.parse.urljoin(base_url, link)
                    if is_internal_link(base_url, absolute_link) and absolute_link not in visited:
                        internal_links.add(absolute_link)
                        to_visit.append(absolute_link)
            except:
                pass

    save_to_csv("website_links.csv", list(internal_links))

base_url = 'https://www.usjr.com.edu.ph'
crawl_website(base_url)


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv

def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            continue

        href = urljoin(url, href)
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

        if not is_valid(href):
            continue

        if href in urls:
            continue

        if domain_name not in href:
            continue

        urls.add(href)

    return urls

if __name__ == "__main__":
    links = get_all_website_links("https://usjr.edu.ph")
    
    with open('usjr_links.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for link in links:
            writer.writerow([link])


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv
from collections import deque

def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_website_links(url):
    urls = set()  # all URLs of the website
    to_visit = deque([url])  # queue of URLs to be processed

    while len(to_visit) != 0:
        print(url)
        url = to_visit.popleft()
        domain_name = urlparse(url).netloc

        try:
            soup = BeautifulSoup(requests.get(url).content, "html.parser")
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
            continue

        for a_tag in soup.findAll("a"):
            href = a_tag.attrs.get("href")
            if href == "" or href is None:
                continue

            href = urljoin(url, href)
            parsed_href = urlparse(href)
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

            if not is_valid(href):
                continue

            if href in urls:
                continue

            if domain_name not in href:
                continue

            urls.add(href)
            to_visit.append(href)

    return urls

if __name__ == "__main__":
    links = get_all_website_links("https://usjr.edu.ph")
    
    with open('usjr_links.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for link in links:
            writer.writerow([link])


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv
from collections import deque

def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme) and parsed.scheme in {"http", "https"}

def get_all_website_links(url):
    urls = set()  # all URLs of the website
    to_visit = deque([url])  # queue of URLs to be processed

    while len(to_visit) != 0:
        url = to_visit.popleft()
        print(url)  # print the current URL
        domain_name = urlparse(url).netloc

        try:
            soup = BeautifulSoup(requests.get(url).content, "html.parser")
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
            continue

        for a_tag in soup.findAll("a"):
            href = a_tag.attrs.get("href")
            if href == "" or href is None:
                continue

            href = urljoin(url, href)
            parsed_href = urlparse(href)
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

            if not is_valid(href):
                continue

            if href in urls:
                continue

            if domain_name not in href:
                continue

            urls.add(href)
            to_visit.append(href)

    return urls

if __name__ == "__main__":
    links = get_all_website_links("https://usjr.edu.ph")
    
    with open('usjr_links.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for link in links:
            writer.writerow([link])


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv
from collections import deque

def is_valid(url):
    parsed = urlparse(url)
    return (bool(parsed.netloc) and bool(parsed.scheme) and parsed.scheme in {"http", "https"} and not 
            (parsed.path.endswith(".png") or parsed.path.endswith(".jpg") or parsed.path.endswith(".jpeg") 
            or parsed.path.endswith(".gif") or parsed.path.endswith(".svg") or parsed.path.endswith(".webp")))

def get_all_website_links(url):
    urls = set()  # all URLs of the website
    to_visit = deque([url])  # queue of URLs to be processed

    with open('usjr_links.csv', 'w', newline='') as file:
        writer = csv.writer(file)

        while len(to_visit) != 0:
            url = to_visit.popleft()
            print(url)  # print the current URL
            domain_name = urlparse(url).netloc

            try:
                soup = BeautifulSoup(requests.get(url).content, "html.parser")
            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, AssertionError):
                continue

            for a_tag in soup.findAll("a"):
                href = a_tag.attrs.get("href")
                if href == "" or href is None:
                    continue

                href = urljoin(url, href)
                parsed_href = urlparse(href)
                href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

                if not is_valid(href):
                    continue

                if href in urls:
                    continue

                if domain_name not in href:
                    continue

                urls.add(href)
                to_visit.append(href)
                writer.writerow([href])

if __name__ == "__main__":
    get_all_website_links("https://usjr.edu.ph")


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv
from collections import deque

def is_valid(url):
    parsed = urlparse(url)
    return (bool(parsed.netloc) and bool(parsed.scheme) and parsed.scheme in {"http", "https"} and not 
            (parsed.path.endswith(".png") or parsed.path.endswith(".jpg") or parsed.path.endswith(".jpeg") 
            or parsed.path.endswith(".gif") or parsed.path.endswith(".svg") or parsed.path.endswith(".webp")))

def get_all_website_links(url):
    urls = set()  # all URLs of the website
    to_visit = deque()  # queue of URLs to be processed

    try:
        with open('visited_links.csv', 'r') as file:
            urls = set(line.strip() for line in file)
        with open('to_visit_links.csv', 'r') as file:
            to_visit = deque(line.strip() for line in file)
    except FileNotFoundError:
        to_visit.append(url)

    with open('visited_links.csv', 'a') as visited_file, open('to_visit_links.csv', 'w') as to_visit_file:
        visited_writer = csv.writer(visited_file)
        to_visit_writer = csv.writer(to_visit_file)

        while len(to_visit) != 0:
            url = to_visit.popleft()
            print(url)  # print the current URL
            domain_name = urlparse(url).netloc

            try:
                soup = BeautifulSoup(requests.get(url).content, "html.parser")
            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, AssertionError):
                continue

            for a_tag in soup.findAll("a"):
                href = a_tag.attrs.get("href")
                if href == "" or href is None:
                    continue

                href = urljoin(url, href)
                parsed_href = urlparse(href)
                href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

                if not is_valid(href):
                    continue

                if href in urls:
                    continue

                if domain_name not in href:
                    continue

                urls.add(href)
                to_visit.append(href)
                visited_writer.writerow([href])

            to_visit_writer.writerows([link] for link in to_visit)

if __name__ == "__main__":
    get_all_website_links("https://usjr.edu.ph")


In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import csv
from collections import deque

def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme) and parsed.scheme in {"http", "https"}

def get_all_website_links(url):
    urls = set()  # all URLs of the website
    pdf_urls = set()  # all PDF URLs of the website
    to_visit = deque()  # queue of URLs to be processed

    try:
        with open('visited_links.csv', 'r') as file:
            urls = set(line.strip() for line in file)
        with open('to_visit_links.csv', 'r') as file:
            to_visit = deque(line.strip() for line in file)
    except FileNotFoundError:
        to_visit.append(url)

    with open('visited_links.csv', 'a') as visited_file, open('to_visit_links.csv', 'w') as to_visit_file, open('pdf_links.csv', 'a') as pdf_file:
        visited_writer = csv.writer(visited_file)
        to_visit_writer = csv.writer(to_visit_file)
        pdf_writer = csv.writer(pdf_file)

        while len(to_visit) != 0:
            url = to_visit.popleft()
            print(url)  # print the current URL
            domain_name = urlparse(url).netloc

            try:
                soup = BeautifulSoup(requests.get(url).content, "html.parser")
            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, AssertionError):
                continue

            for a_tag in soup.findAll("a"):
                href = a_tag.attrs.get("href")
                if href == "" or href is None:
                    continue

                href = urljoin(url, href)
                parsed_href = urlparse(href)
                href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

                if not is_valid(href):
                    continue

                if domain_name not in href:
                    continue

                if href.endswith(".pdf"):
                    if href not in pdf_urls:
                        pdf_urls.add(href)
                        pdf_writer.writerow([href])
                else:
                    if href in urls:
                        continue
                    urls.add(href)
                    to_visit.append(href)
                    visited_writer.writerow([href])

            to_visit_writer.writerows([link] for link in to_visit)

if __name__ == "__main__":
    get_all_website_links("https://usjr.edu.ph")


https://usjr.edu.ph
https://usjr.edu.ph/
https://usjr.edu.ph/about/
https://usjr.edu.ph/admissions/
https://usjr.edu.ph/academics/
https://usjr.edu.ph/sas/
https://usjr.edu.ph/sbm/
https://usjr.edu.ph/soe/
https://usjr.edu.ph/soeng/
https://usjr.edu.ph/scs/
https://usjr.edu.ph/sams/
https://usjr.edu.ph/law/
https://usjr.edu.ph/elementary/
https://usjr.edu.ph/juniorhighschool/
https://usjr.edu.ph/seniorhighschool/
https://usjr.edu.ph/rittc/
https://usjr.edu.ph/student-life/
https://usjr.edu.ph/sdpc/
https://usjr.edu.ph/library/
https://usjr.edu.ph/library/branch-libraries/library-elementary/
https://usjr.edu.ph/library/branch-libraries/library-junior-high-school/
https://usjr.edu.ph/library/branch-libraries/senor-high-school/
https://usjr.edu.ph/library/branch-libraries/library-imc/
https://usjr.edu.ph/cmo/
https://usjr.edu.ph/research/
https://usjr.edu.ph/alumni/
https://usjr.edu.ph/online-enrollment/
https://eclass.coe.usjr.edu.ph
https://eclass.cre.usjr.edu.ph
https://eclass.commerce

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://usjr.edu.ph/wp-content/uploads/2016/05/law-kunuhay-600x400.png


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://usjr.edu.ph/wp-content/uploads/2016/05/gradeschool-for-sure-600x400.png


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://usjr.edu.ph/wp-content/uploads/2016/11/SAM_4368.jpg


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://usjr.edu.ph/wp-content/uploads/2016/11/SAM_4408.jpg


KeyboardInterrupt: 